<a href="https://colab.research.google.com/github/palaciosestefania/T-picos-en-Ecuaciones-Diferenciales/blob/main/EDO_Transformada_de_Laplace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np
import sympy as sp

In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
from sympy import symbols, laplace_transform
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.sans-serif'] = 'stix'

In [5]:
import sympy
sympy.init_printing()

In [6]:
from scipy import integrate

In [7]:
t = sympy.symbols("t", positive=True)

In [8]:
s, Y = sympy.symbols("s, Y", real=True)

In [9]:
y = sympy.Function("y")

In [10]:
ode = y(t).diff(t, 2) + 2 * y(t).diff(t) + 10 * y(t) - 2 * sympy.sin(3*t)

In [11]:
ode

                                     2       
                         d          d        
10⋅y(t) - 2⋅sin(3⋅t) + 2⋅──(y(t)) + ───(y(t))
                         dt           2      
                                    dt       

In [12]:
def L(ode):
    return sympy.laplace_transform(ode, t, s, noconds=True)

In [13]:
L(ode)

 2                                                                                                 ↪
s ⋅LaplaceTransform(y(t), t, s) + 2⋅s⋅LaplaceTransform(y(t), t, s) - s⋅y(0) + 10⋅LaplaceTransform( ↪
                                                                                                   ↪
                                                                                                   ↪

↪                        ⎛d       ⎞│        6   
↪ y(t), t, s) - 2⋅y(0) - ⎜──(y(t))⎟│    - ──────
↪                        ⎝dt      ⎠│t=0    2    
↪                                         s  + 9

In [14]:
def laplace_transform_derivatives(e):
    """
    Evaluate the laplace transforms of derivatives of functions
    """
    if isinstance(e, sympy.LaplaceTransform):
        if isinstance(e.args[0], sympy.Derivative):
            d, t, s = e.args
            n = d.args[1][1]
            return ((s**n) * sympy.LaplaceTransform(d.args[0], t, s) -
                    sum([s**(n-i) * sympy.diff(d.args[0], t, i-1).subs(t, 0)
                         for i in range(1, n+1)]))

    if isinstance(e, (sympy.Add, sympy.Mul)):
        t = type(e)
        return t(*[laplace_transform_derivatives(arg) for arg in e.args])

    return e

In [15]:
L1=laplace_transform_derivatives(L(ode))

In [16]:
L1

 2                                                                                                 ↪
s ⋅LaplaceTransform(y(t), t, s) + 2⋅s⋅LaplaceTransform(y(t), t, s) - s⋅y(0) + 10⋅LaplaceTransform( ↪
                                                                                                   ↪
                                                                                                   ↪

↪                        ⎛d       ⎞│        6   
↪ y(t), t, s) - 2⋅y(0) - ⎜──(y(t))⎟│    - ──────
↪                        ⎝dt      ⎠│t=0    2    
↪                                         s  + 9

In [17]:
L_y = sympy.laplace_transform(y(t), t, s)

In [18]:
L_y

(LaplaceTransform(y(t), t, s), -∞, True)

In [19]:
L3=L1.subs({L_y: Y})
L3

 2                                                                                                 ↪
s ⋅LaplaceTransform(y(t), t, s) + 2⋅s⋅LaplaceTransform(y(t), t, s) - s⋅y(0) + 10⋅LaplaceTransform( ↪
                                                                                                   ↪
                                                                                                   ↪

↪                        ⎛d       ⎞│        6   
↪ y(t), t, s) - 2⋅y(0) - ⎜──(y(t))⎟│    - ──────
↪                        ⎝dt      ⎠│t=0    2    
↪                                         s  + 9

In [21]:
equation = L1
result = equation.subs(L_y, Y)
result

 2                                                                                                 ↪
s ⋅LaplaceTransform(y(t), t, s) + 2⋅s⋅LaplaceTransform(y(t), t, s) - s⋅y(0) + 10⋅LaplaceTransform( ↪
                                                                                                   ↪
                                                                                                   ↪

↪                        ⎛d       ⎞│        6   
↪ y(t), t, s) - 2⋅y(0) - ⎜──(y(t))⎟│    - ──────
↪                        ⎝dt      ⎠│t=0    2    
↪                                         s  + 9

In [22]:
ics = {y(0): 1, y(t).diff(t).subs(t, 0): 0}

In [23]:
ics

⎧         ⎛d       ⎞│      ⎫
⎨y(0): 1, ⎜──(y(t))⎟│   : 0⎬
⎩         ⎝dt      ⎠│t=0   ⎭

In [24]:
L4=L3.subs(ics)
L4

 2                                                                                                 ↪
s ⋅LaplaceTransform(y(t), t, s) + 2⋅s⋅LaplaceTransform(y(t), t, s) - s + 10⋅LaplaceTransform(y(t), ↪
                                                                                                   ↪
                                                                                                   ↪

↪                6   
↪  t, s) - 2 - ──────
↪               2    
↪              s  + 9

In [32]:
Y_sol=sympy.solve(L4,L_y)[0]
Y_sol

⎛     3      2                          ⎞
⎜    s  + 2⋅s  + 9⋅s + 24               ⎟
⎜─────────────────────────────, -∞, True⎟
⎜ 4      3       2                      ⎟
⎝s  + 2⋅s  + 19⋅s  + 18⋅s + 90          ⎠

In [33]:
#Y_sol = sympy.solve(L4,Y)
#_sol

In [34]:
sympy.apart(Y_sol[0])

  6⋅(2⋅s - 1)       49⋅s + 92     
- ─────────── + ──────────────────
     ⎛ 2    ⎞      ⎛ 2           ⎞
  37⋅⎝s  + 9⎠   37⋅⎝s  + 2⋅s + 10⎠

In [35]:
def invL(F):
    return sympy.inverse_laplace_transform(F, s, t)
    #Se declara a la inversa de la transfromada de Laplace como L(f) para así solo "llamarla"

In [38]:
y_sol = invL(Y_sol[0])
y_sol

                               -t                -t         
2⋅sin(3⋅t)   12⋅cos(3⋅t)   43⋅ℯ  ⋅sin(3⋅t)   49⋅ℯ  ⋅cos(3⋅t)
────────── - ─────────── + ─────────────── + ───────────────
    37           37              111               37       

In [39]:
YS=sympy.simplify(y_sol)
YS

⎛                           t                             ⎞  -t
⎝6⋅(sin(3⋅t) - 6⋅cos(3⋅t))⋅ℯ  + 43⋅sin(3⋅t) + 147⋅cos(3⋅t)⎠⋅ℯ  
───────────────────────────────────────────────────────────────
                              111                              

In [40]:
sympy.simplify(y_sol).evalf()

                    ⎛                               t                                 ⎞  -t
0.00900900900900901⋅⎝6.0⋅(sin(3⋅t) - 6.0⋅cos(3⋅t))⋅ℯ  + 43.0⋅sin(3⋅t) + 147.0⋅cos(3⋅t)⎠⋅ℯ  

In [41]:
y_t = sympy.lambdify(t, y_sol.evalf(), 'numpy')

In [42]:
y_t

<function _lambdifygenerated(t)>

In [59]:
#%version_information numpy, scipy, sympy, matplotlib